# Text Summarization and Topic Models
* Text Summarization and Information Extraction
* Important Concepts
* Keyphrase Extractions
    1. Collocations
    2. Weighted Tag-Based Phrase Extraction
* Topic Modeling on Research Papers
    1. The Main Objective
    2. Data Retrieval
    3. Load and View Dataset
    4. Basic Text Wrangling
* Topic Models with Gensim
    1. Text Representation with Feature Engineering
    2. Latent Semantic Indexing
    3. Implementing LSI Topic Models from Scratch
    4. Latent Dirichlet Allocation
    5. LDA Models with MALLET
    6. LDA Tuning: Finding the Optimal Number of Topics
    7. Interpreting Topic Model Results
    8. Predicting Topics for New Research Papers
* Topic Models with Scikit-Learn
    1. Text Representation with Feature Engineering
    2. Latent Semantic Indexing
    3. Latent Dirichlet Allocation
    4. Non-Negative Matrix Factorization
    5. Predicting Topics for New Research Papers
    6. Visualizing Topic Models
* Automated Document Summarization
    1. Text Wrangling
    2. Text Representation with Feature Engineering
    3. Latent Semantic Analysis
    4. TextRank

In [7]:
# if spacy doesn't run
#!pip3 install spacy
#!python -m spacy download en_core_web_sm

In [8]:
# if nltk error
#import nltk
#nltk.download('all')

## Important Concepts

In [1]:
# extract top k singular values and return corresponding U, S, & V matrices
from scipy.sparse.linalg import svds

def low_rank_svd(matrix, singular_count=2):
    u,s,vt = svds(matrix, k=singular_count)
    return u,s,vt

## Keyphrase Extraction

In [9]:
## Collocations
from nltk.corpus import gutenberg
import text_normalizer as tn
import nltk
from operator import itemgetter

# load corpus
alice = gutenberg.sents(fileids='carroll-alice.txt')
alice = [' '.join(ts) for ts in alice]
norm_alice = list(filter(None,
                         tn.normalize_corpus(alice, text_lemmatization=False)))

# print and compare first line
print(alice[0], '\n', norm_alice[0])

[ Alice ' s Adventures in Wonderland by Lewis Carroll 1865 ] 
 alice adventures wonderland lewis carroll


In [12]:
def compute_ngrams(sequence, n):
    return list(
            zip(*(sequence[index:]
                  for index in range(n))))

# test function
compute_ngrams([1,2,3,4], 2) # bi-grams
compute_ngrams([1,2,3,4], 3) # tri-grams

[(1, 2, 3), (2, 3, 4)]

In [13]:
# function to flatten corpus into one big string of text
def flatten_corpus(corpus):
    return ' '.join([document.strip()
                    for document in corpus])

# get top n-grams for corpus of text
def get_top_ngrams(corpus, ngram_val=1, limit=5):
    corpus = flatten_corpus(corpus)
    tokens = nltk.word_tokenize(corpus)
    
    ngrams = compute_ngrams(tokens, ngram_val)
    ngrams_freq_dist = nltk.FreqDist(ngrams)
    sorted_ngrams_fd = sorted(ngrams_freq_dist.items(),
                              key=itemgetter(1), reverse=True)
    sorted_ngrams = sorted_ngrams_fd[0:limit]
    sorted_ngrams = [(' '.join(text), freq)
                     for text, freq in sorted_ngrams]
    return sorted_ngrams

In [14]:
# top 10 bigrams
get_top_ngrams(corpus=norm_alice, ngram_val=2, limit=10)

[('said alice', 123),
 ('mock turtle', 56),
 ('march hare', 31),
 ('said king', 29),
 ('thought alice', 26),
 ('white rabbit', 22),
 ('said hatter', 22),
 ('said mock', 20),
 ('said caterpillar', 18),
 ('said gryphon', 18)]

In [15]:
# top 10 trigrams
get_top_ngrams(corpus=norm_alice, ngram_val=3, limit=10)

[('said mock turtle', 20),
 ('said march hare', 10),
 ('poor little thing', 6),
 ('little golden key', 5),
 ('certainly said alice', 5),
 ('white kid gloves', 5),
 ('march hare said', 5),
 ('mock turtle said', 5),
 ('know said alice', 4),
 ('might well say', 4)]

In [16]:
# use NLTK's collocation finders
# bigrams
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import BigramAssocMeasures

finder = BigramCollocationFinder.from_documents([item.split() for item in norm_alice])
finder

In [20]:
bigram_measures = BigramAssocMeasures()

# raw frequencies
finder.nbest(bigram_measures.raw_freq, 10)

[('said', 'alice'),
 ('mock', 'turtle'),
 ('march', 'hare'),
 ('said', 'king'),
 ('thought', 'alice'),
 ('said', 'hatter'),
 ('white', 'rabbit'),
 ('said', 'mock'),
 ('said', 'caterpillar'),
 ('said', 'gryphon')]

In [21]:
# pointwise mutual information
finder.nbest(bigram_measures.pmi, 10)

[('abide', 'figures'),
 ('acceptance', 'elegant'),
 ('accounting', 'tastes'),
 ('accustomed', 'usurpation'),
 ('act', 'crawling'),
 ('adjourn', 'immediate'),
 ('adoption', 'energetic'),
 ('affair', 'trusts'),
 ('agony', 'terror'),
 ('alarmed', 'proposal')]

In [22]:
# trigrams
from nltk.collocations import TrigramCollocationFinder
from nltk.collocations import TrigramAssocMeasures

finder = TrigramCollocationFinder.from_documents([item.split() for item in norm_alice])

trigram_measures = TrigramAssocMeasures()

In [23]:
# raw frequencies
finder.nbest(trigram_measures.raw_freq, 10)

[('said', 'mock', 'turtle'),
 ('said', 'march', 'hare'),
 ('poor', 'little', 'thing'),
 ('little', 'golden', 'key'),
 ('march', 'hare', 'said'),
 ('mock', 'turtle', 'said'),
 ('white', 'kid', 'gloves'),
 ('beau', 'ootiful', 'soo'),
 ('certainly', 'said', 'alice'),
 ('might', 'well', 'say')]

In [24]:
# pointwise mutual information
finder.nbest(trigram_measures.pmi, 10)

[('accustomed', 'usurpation', 'conquest'),
 ('adjourn', 'immediate', 'adoption'),
 ('adoption', 'energetic', 'remedies'),
 ('ancient', 'modern', 'seaography'),
 ('apple', 'roast', 'turkey'),
 ('arithmetic', 'ambition', 'distraction'),
 ('brother', 'latin', 'grammar'),
 ('canvas', 'bag', 'tied'),
 ('cherry', 'tart', 'custard'),
 ('circle', 'exact', 'shape')]

In [25]:
## Weighted Tag-Based Phrase Extraction
data = open('elephants.txt', 'r+').readlines()
sentences = nltk.sent_tokenize(data[0])
len(sentences)

29

In [26]:
# viewing the first three lines
sentences[:3]

['Elephants are large mammals of the family Elephantidae and the order Proboscidea.',
 'Three species are currently recognised: the African bush elephant (Loxodonta africana), the African forest elephant (L. cyclotis), and the Asian elephant (Elephas maximus).',
 'Elephants are scattered throughout sub-Saharan Africa, South Asia, and Southeast Asia.']

In [27]:
norm_sentences = tn.normalize_corpus(sentences, text_lower_case=False, text_stemming=False,
                                     text_lemmatization=False, stopword_removal=False)
norm_sentences[:3]

['Elephants are large mammals of the family Elephantidae and the order Proboscidea',
 'Three species are currently recognised the African bush elephant Loxodonta africana the African forest elephant L cyclotis and the Asian elephant Elephas maximus',
 'Elephants are scattered throughout subSaharan Africa South Asia and Southeast Asia']

In [34]:
import itertools
stopwords = nltk.corpus.stopwords.words('english')

def get_chunks(sentences, grammar=r'NP: {<DT>? <JJ>* <NN.*>+}', stopword_list=stopwords):
    all_chunks = []
    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    
    for sentence in sentences:
        tagged_sents = [nltk.pos_tag(nltk.word_tokenize(sentence))]
        chunks = [chunker.parse(tagged_sent)
                     for tagged_sent in tagged_sents]
        wtc_sents = [nltk.chunk.tree2conlltags(chunk)
                        for chunk in chunks]
        flattened_chunks = list(itertools.chain.from_iterable(wtc_sent for wtc_sent in wtc_sents))
        valid_chunks_tagged = [(status, [wtc for wtc in chunk])
                                    for status, chunk in itertools.groupby(flattened_chunks,
                                                      lambda word_pos_chunk: 
                                                      word_pos_chunk[2] != 'O')]
        valid_chunks = [' '.join(word.lower()
                                 for word, tag, chunk in wtc_group
                                     if word.lower() not in stopword_list)
                                        for status, wtc_group in valid_chunks_tagged if status]
        all_chunks.append(valid_chunks)
    return all_chunks

In [35]:
chunks = get_chunks(norm_sentences)
chunks

[['elephants', 'large mammals', 'family elephantidae', 'order proboscidea'],
 ['species',
  'african bush elephant loxodonta',
  'african forest elephant l cyclotis',
  'asian elephant elephas maximus'],
 ['elephants', 'subsaharan africa south asia', 'southeast asia'],
 ['elephantidae',
  'family',
  'order proboscidea',
  'extinct members',
  'order',
  'deinotheres gomphotheres mammoths',
  'mastodons'],
 ['elephants',
  'several distinctive features',
  'long trunk',
  'proboscis',
  'many purposes',
  'water',
  'grasping objects'],
 ['incisors', 'tusks', 'weapons', 'tools', 'objects'],
 ['elephants', 'flaps', 'body temperature'],
 ['pillarlike legs', 'great weight'],
 ['african elephants',
  'ears',
  'backs',
  'asian elephants',
  'ears',
  'convex',
  'level backs'],
 ['elephants', 'different habitats', 'savannahs forests deserts', 'marshes'],
 ['water'],
 ['keystone species', 'impact', 'environments'],
 ['animals',
  'distance',
  'elephants',
  'predators',
  'lions tigers hy

In [42]:
from gensim import corpora, models

def get_tfidf_weighted_keyphrases(sentences, grammar=r'NP: {<DT>? <JJ>* <NN.*>+}', top_n=10):
    valid_chunks = get_chunks(sentences, grammar=grammar)
    
    dictionary = corpora.Dictionary(valid_chunks)
    corpus = [dictionary.doc2bow(chunk) for chunk in valid_chunks]
    
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    
    weighted_phrases = {dictionary.get(idx): value for doc in corpus_tfidf for idx, value in doc}
    weighted_phrases = sorted(weighted_phrases.items(),
                              key=itemgetter(1), reverse=True)
    weighted_phrases = [(term, round(wt,3)) for term, wt in weighted_phrases]
    
    return weighted_phrases[:top_n]

In [43]:
# top 30 tf-idf weighted keyphrases
get_tfidf_weighted_keyphrases(sentences=norm_sentences, top_n=30)

[('water', 1.0),
 ('asia', 0.807),
 ('wild', 0.764),
 ('great weight', 0.707),
 ('pillarlike legs', 0.707),
 ('southeast asia', 0.693),
 ('subsaharan africa south asia', 0.693),
 ('body temperature', 0.693),
 ('flaps', 0.693),
 ('fissionfusion society', 0.693),
 ('multiple family groups', 0.693),
 ('art folklore religion literature', 0.693),
 ('popular culture', 0.693),
 ('ears', 0.681),
 ('males', 0.653),
 ('males bulls', 0.653),
 ('family elephantidae', 0.607),
 ('large mammals', 0.607),
 ('years', 0.607),
 ('environments', 0.577),
 ('impact', 0.577),
 ('keystone species', 0.577),
 ('cetaceans', 0.577),
 ('elephant intelligence', 0.577),
 ('primates', 0.577),
 ('dead individuals', 0.577),
 ('kind', 0.577),
 ('selfawareness', 0.577),
 ('different habitats', 0.57),
 ('marshes', 0.57)]

In [44]:
from gensim.summarization import keywords

key_words = keywords(data[0], ratio=1.0, scores=True, lemmatize=True)
[(item, round(score,3)) for item, score in key_words][:25]

[('african bush elephant', 0.261),
 ('including', 0.141),
 ('family', 0.137),
 ('cow', 0.124),
 ('forests', 0.108),
 ('female', 0.103),
 ('asia', 0.102),
 ('objects', 0.098),
 ('sight', 0.098),
 ('ivory', 0.098),
 ('tigers', 0.098),
 ('males', 0.088),
 ('folklore', 0.087),
 ('religion', 0.087),
 ('known', 0.087),
 ('larger ears', 0.085),
 ('water', 0.075),
 ('highly recognisable', 0.075),
 ('breathing lifting', 0.074),
 ('flaps', 0.073),
 ('africa', 0.072),
 ('gomphotheres', 0.072),
 ('animals tend', 0.071),
 ('success', 0.071),
 ('south', 0.07)]

## Topic Modeling on Research Papers

In [1]:
## Data Retrieval
!wget https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz

--2020-05-10 17:42:09--  https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz
Resolving cs.nyu.edu (cs.nyu.edu)... 128.122.49.30
Connecting to cs.nyu.edu (cs.nyu.edu)|128.122.49.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12851423 (12M) [application/x-gzip]
Saving to: ‘nips12raw_str602.tgz’

nips12raw_str602.tg 100%[===================>]  12.26M  23.3MB/s    in 0.5s    

2020-05-10 17:42:09 (23.3 MB/s) - ‘nips12raw_str602.tgz’ saved [12851423/12851423]



In [2]:
# extract dataset
!tar -xzf nips12raw_str602.tgz

In [3]:
import os
import numpy as np
import pandas as pd

DATA_PATH = 'nipstxt/'
print(os.listdir(DATA_PATH))

['nips11', 'nips10', 'nips03', 'README_yann', 'MATLAB_NOTES', 'nips08', 'nips00', 'nips06', 'nips05', 'nips12', 'nips02', 'nips04', 'idx', 'orig', 'nips07', 'nips09', 'nips01', 'RAW_DATA_NOTES']


In [5]:
## Load and View Dataset
folders = ["nips{0:02}".format(i) for i in range(0,13)]
# read all texts into a list
papers = []
for folder in folders:
    file_names = os.listdir(DATA_PATH + folder)
    for file_name in file_names:
        with open(DATA_PATH + folder + '/' + file_name, encoding='utf-8', errors='ignore', mode='r+') as f:
            data = f.read()
        papers.append(data)
len(papers)

1740

In [7]:
print(papers[0][:1000])

814 
NEU1OMO1PHIC NETWORKS BASED 
ON SPARSE OPTICAL ORTHOGONAL CODES 
Mario P. Vecchi and Jawad A. Salehl 
Bell Communications Research 
435 South Street 
Morristown, NJ 07960-1961 
Abstract 
A family of neuromorphic networks specifically designed for communications 
and optical signal processing applications is presented. The information is encoded 
utilizing sparse Optical Orthogonal Code sequences on the basis of unipolar, binary 
(0, 1) signals. The generalized synaptic connectivity matrix is also unipolar, and 
clipped to binary (0, 1) values. In addition to high-capacity associative memory, 
the resulting neural networks can be used to implement general functions, such as 
code filtering, code mapping, code joining, code shifting and code projecting. 
1 Introduction 
Synthetic neural nets [1,2] represent an active and growing research field. Fundamental 
issues, as well as practical implementations with electronic and optical devices are being 
studied. In addition, several lea

In [11]:
%%time
## Basic Text Wrangling
import nltk

stop_words = nltk.corpus.stopwords.words('english')
wtk = nltk.tokenize.RegexpTokenizer(r'\w+')
wnl = nltk.stem.wordnet.WordNetLemmatizer()

def normalize_corpus(papers):
    norm_papers = []
    for paper in papers:
        paper = paper.lower()
        paper_tokens = [token.strip() for token in wtk.tokenize(paper)]
        paper_tokens = [wnl.lemmatize(token) for token in paper_tokens if not token.isnumeric()]
        paper_tokens = [token for token in paper_tokens if len(token) > 1]
        paper_tokens = [token for token in paper_tokens if token not in stop_words]
        paper_tokens = list(filter(None, paper_tokens))
        if paper_tokens:
            norm_papers.append(paper_tokens)
        
    return norm_papers

norm_papers = normalize_corpus(papers)
print(len(norm_papers))

1740
CPU times: user 28.7 s, sys: 179 ms, total: 28.9 s
Wall time: 29.1 s


In [12]:
# viewing a processed paper
print(norm_papers[0][:50])

['neu1', 'omo1', 'phic', 'network', 'based', 'sparse', 'optical', 'orthogonal', 'code', 'mario', 'vecchi', 'jawad', 'salehl', 'bell', 'communication', 'research', 'south', 'street', 'morristown', 'nj', 'abstract', 'family', 'neuromorphic', 'network', 'specifically', 'designed', 'communication', 'optical', 'signal', 'processing', 'application', 'presented', 'information', 'encoded', 'utilizing', 'sparse', 'optical', 'orthogonal', 'code', 'sequence', 'basis', 'unipolar', 'binary', 'signal', 'generalized', 'synaptic', 'connectivity', 'matrix', 'also', 'unipolar']


## Topic Models with Gensim

In [15]:
## Text Representation with Feature Engineering
import gensim

bigram = gensim.models.Phrases(norm_papers, min_count=20, 
                               threshold=20, delimiter=b'_') # higher threshold fewer phrases
bigram_model = gensim.models.phrases.Phraser(bigram)

# sample demonstration
print(bigram_model[norm_papers[0]][:50])

['neu1', 'omo1', 'phic', 'network', 'based', 'sparse', 'optical', 'orthogonal', 'code', 'mario', 'vecchi', 'jawad', 'salehl', 'bell', 'communication', 'research', 'south', 'street', 'morristown', 'nj_abstract', 'family', 'neuromorphic', 'network', 'specifically', 'designed', 'communication', 'optical', 'signal_processing', 'application', 'presented', 'information', 'encoded', 'utilizing', 'sparse', 'optical', 'orthogonal', 'code', 'sequence', 'basis', 'unipolar', 'binary', 'signal', 'generalized', 'synaptic', 'connectivity', 'matrix', 'also', 'unipolar', 'clipped', 'binary']


In [16]:
norm_corpus_bigrams = [bigram_model[doc] for doc in norm_papers]

# create a dictionary representation of the documents
dictionary = gensim.corpora.Dictionary(norm_corpus_bigrams)
print('Sample word to number mappings:', list(dictionary.items())[:15])
print('Total Vocabulary Size', len(dictionary))

Sample word to number mappings: [(0, '2f'), (1, '2our'), (2, '3the'), (3, '82o'), (4, '86ch'), (5, '_b'), (6, '_t'), (7, '_u'), (8, '_v'), (9, '_ym'), (10, '_z'), (11, 'ability'), (12, 'absence'), (13, 'ac'), (14, 'acad_sci')]
Total Vocabulary Size 78892


In [17]:
# filer out words that occur less than 20 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=20, no_above=0.6)
print('Total Vocabulary Size:', len(dictionary))

Total Vocabulary Size: 7756


In [18]:
# transforming corpus into bag of words vectors
bow_corpus = [dictionary.doc2bow(text) for text in norm_corpus_bigrams]
print(bow_corpus[1][:50])

[(2, 1), (8, 1), (13, 2), (20, 1), (24, 1), (34, 2), (46, 2), (52, 1), (56, 1), (57, 2), (58, 2), (66, 1), (76, 3), (77, 1), (83, 1), (86, 1), (87, 1), (100, 1), (101, 1), (103, 1), (105, 2), (108, 1), (113, 1), (117, 1), (118, 2), (119, 1), (126, 1), (130, 1), (132, 1), (138, 2), (148, 2), (155, 1), (169, 2), (172, 1), (173, 3), (177, 2), (185, 2), (186, 2), (189, 2), (190, 1), (200, 1), (210, 3), (211, 1), (217, 1), (218, 3), (222, 4), (227, 5), (231, 2), (240, 2), (245, 1)]


In [19]:
# viewing actual terms and their counts
print([(dictionary[idx], freq) for idx, freq in bow_corpus[1][:50]])

[('absence', 1), ('accurately', 1), ('addition', 2), ('american_institute', 1), ('application', 1), ('available', 2), ('body', 2), ('channel', 1), ('class', 1), ('clear', 2), ('clearly', 2), ('combination', 1), ('computational', 3), ('computer', 1), ('condition', 1), ('connected', 1), ('connectivity', 1), ('corresponding', 1), ('corresponds', 1), ('cross', 1), ('cycle', 2), ('defined', 1), ('design', 1), ('determined', 1), ('developed', 2), ('development', 1), ('directly', 1), ('discussion', 1), ('distinct', 1), ('easily', 2), ('end', 2), ('established', 1), ('far', 2), ('feedback', 1), ('fiber', 3), ('filter', 2), ('functional', 2), ('fundamental', 2), ('generally', 2), ('generate', 1), ('hard', 1), ('iii', 3), ('ij', 1), ('importance', 1), ('important', 3), ('indicated', 4), ('intensity', 5), ('interesting', 2), ('le', 2), ('line', 1)]


In [20]:
# total papers in the corpus
print('Total number of papers:', len(bow_corpus))

Total number of papers: 1740


In [22]:
%%time
## Latent Semantic Indexing
TOTAL_TOPICS = 10
lsi_bow = gensim.models.LsiModel(bow_corpus, id2word=dictionary, num_topics=TOTAL_TOPICS, 
                                 onepass=True, chunksize=1740, power_iters=1000)

CPU times: user 2min 37s, sys: 39.9 ms, total: 2min 37s
Wall time: 2min 37s


In [23]:
for topic_id, topic in lsi_bow.print_topics(num_topics=10, num_words=20):
    print('Topic #'+str(topic_id+1)+':')
    print(topic)
    print()

Topic #1:
0.215*"unit" + 0.212*"state" + 0.187*"training" + 0.177*"neuron" + 0.162*"pattern" + 0.145*"image" + 0.140*"vector" + 0.125*"feature" + 0.122*"cell" + 0.110*"layer" + 0.101*"task" + 0.097*"class" + 0.091*"probability" + 0.089*"signal" + 0.087*"step" + 0.086*"response" + 0.085*"representation" + 0.083*"noise" + 0.082*"rule" + 0.081*"distribution"

Topic #2:
0.487*"neuron" + 0.396*"cell" + -0.257*"state" + 0.191*"response" + -0.187*"training" + 0.170*"stimulus" + 0.117*"activity" + -0.109*"class" + 0.099*"spike" + 0.097*"pattern" + 0.096*"circuit" + 0.096*"synaptic" + -0.095*"vector" + 0.090*"signal" + 0.090*"firing" + 0.088*"visual" + -0.084*"classifier" + -0.083*"action" + -0.078*"word" + 0.078*"cortical"

Topic #3:
-0.627*"state" + 0.395*"image" + -0.219*"neuron" + 0.209*"feature" + -0.188*"action" + 0.137*"unit" + 0.131*"object" + -0.130*"control" + 0.129*"training" + -0.109*"policy" + 0.103*"classifier" + 0.090*"class" + -0.081*"step" + -0.081*"dynamic" + 0.080*"classifica

In [25]:
for n in range(TOTAL_TOPICS):
    print('Topic #'+str(n+1)+':')
    print('='*50)
    d1 = []
    d2 = []
    for term, wt in lsi_bow.show_topic(n, topn=20):
        if wt >= 0:
            d1.append((term, round(wt,3)))
        else:
            d2.append((term, round(wt,3)))
    print('Direction 1:', d1)
    print('-'*50)
    print('Direction 2:', d2)
    print('-'*50)
    print()

Topic #1:
Direction 1: [('unit', 0.215), ('state', 0.212), ('training', 0.187), ('neuron', 0.177), ('pattern', 0.162), ('image', 0.145), ('vector', 0.14), ('feature', 0.125), ('cell', 0.122), ('layer', 0.11), ('task', 0.101), ('class', 0.097), ('probability', 0.091), ('signal', 0.089), ('step', 0.087), ('response', 0.086), ('representation', 0.085), ('noise', 0.083), ('rule', 0.082), ('distribution', 0.081)]
--------------------------------------------------
Direction 2: []
--------------------------------------------------

Topic #2:
Direction 1: [('neuron', 0.487), ('cell', 0.396), ('response', 0.191), ('stimulus', 0.17), ('activity', 0.117), ('spike', 0.099), ('pattern', 0.097), ('circuit', 0.096), ('synaptic', 0.096), ('signal', 0.09), ('firing', 0.09), ('visual', 0.088), ('cortical', 0.078)]
--------------------------------------------------
Direction 2: [('state', -0.257), ('training', -0.187), ('class', -0.109), ('vector', -0.095), ('classifier', -0.084), ('action', -0.083), ('w

In [26]:
# get U, S, VT matrices from topic model
term_topic = lsi_bow.projection.u
singular_values = lsi_bow.projection.s
topic_document = (gensim.matutils.corpus2dense(lsi_bow[bow_corpus], len(singular_values)).T / singular_values).T
term_topic.shape, singular_values.shape, topic_document.shape

((7756, 10), (10,), (10, 1740))

In [27]:
# document topic matrix for our LSI model
document_topics = pd.DataFrame(np.round(topic_document.T, 3), 
                               columns=['T'+str(i) for i in range(1, TOTAL_TOPICS+1)])
document_topics.head(5)

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10
0,0.025,-0.011,-0.001,0.011,0.026,0.015,0.030,-0.025,-0.016,0.073
1,0.021,0.028,-0.004,0.003,-0.003,-0.000,-0.008,0.009,0.008,-0.014
2,0.027,0.007,0.006,0.017,0.017,0.035,-0.008,-0.001,-0.079,0.075
3,0.016,0.017,-0.013,0.008,0.024,0.028,0.000,-0.019,0.008,-0.006
4,0.018,0.030,-0.003,0.001,0.006,-0.031,-0.005,-0.001,0.001,0.004


In [28]:
document_numbers = [13, 250, 500]
for document_number in document_numbers:
    top_topics = list(document_topics.
                      columns[np.argsort(-np.absolute(document_topics.iloc[document_number].values))[:3]])
    print('Document #'+str(document_number)+':')
    print('Dominant Topics (top 3):', top_topics)
    print('Paper Summary:')
    print(papers[document_number][:500])
    print()

Document #13:
Dominant Topics (top 3): ['T8', 'T1', 'T4']
Paper Summary:
412 
CAPACITY FOR PATTERNS AND SEQUENCES IN KANERVA'S SDM 
AS COMPARED TO OTHER ASSOCIATIVE MEMORY MODELS 
James D. Keeler 
Chemistry Department, Stanford University, Stanford, CA 94305 
and RIACS, NASA-AMES 230-5 Moffett Field, CA 94035. 
e.rnail: jdk hydra.riacs. edu 
ABSTRACT 
The information capacity of Kanerva's Sparse, Distributed Memory (SDM) and Hopfield-type 
neural networks is investigated. Under the approximations used here, it is shown that the to- 
tal information stored in these s

Document #250:
Dominant Topics (top 3): ['T8', 'T1', 'T2']
Paper Summary:
68 Baird 
Associative Memory in a Simple Model of 
Oscillating Cortex 
Bill Baird 
Dept Molecular and Cell Biology, 
U.C.Berkeley, Berkeley, Ca. 94720 
ABSTRACT 
A generic model of oscillating cortex, which assumes "minimal" 
coupling justified by known anatomy, is shown to function as an 
sociative memory, using previously developed theory. The net

In [29]:
## Implementing LSI Topic Models from Scratch
td_matrix = gensim.matutils.corpus2dense(corpus=bow_corpus, num_terms=len(dictionary))
print(td_matrix.shape)
td_matrix

(7756, 1740)


array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [30]:
vocabulary = np.array(list(dictionary.values()))
print('Total vocabulary size:', len(vocabulary))
vocabulary

Total vocabulary size: 7756


array(['3the', 'ability', 'absence', ..., 'smola', 'mozer_jordan',
       'kearns_solla'], dtype='<U28')

In [31]:
from scipy.sparse.linalg import svds
u, s, vt = svds(td_matrix, k=TOTAL_TOPICS, maxiter=10000)
term_topic = u
singular_values = s
topic_document = vt
term_topic.shape, singular_values.shape, topic_document.shape

((7756, 10), (10,), (10, 1740))

In [32]:
tt_weights = term_topic.transpose() * singular_values[:,None]
tt_weights.shape

(10, 7756)

In [33]:
top_terms = 20
topic_key_term_idxs = np.argsort(-np.absolute(tt_weights), axis=1)[:, :top_terms]
topic_keyterm_weights = np.array([tt_weights[row, columns]
                                 for row, columns in list(zip(np.arange(TOTAL_TOPICS), topic_key_term_idxs))])
topic_keyterms = vocabulary[topic_key_term_idxs]
topic_keyterms_weights = list(zip(topic_keyterms, topic_keyterm_weights))
for n in range(TOTAL_TOPICS):
    print('Topic #'+str(n+1)+':')
    print('='*50)
    d1 = []
    d2 = []
    terms, weights = topic_keyterms_weights[n]
    term_weights = sorted([(t,w) for t, w in zip(terms, weights)], key=lambda row: -abs(row[1]))
    for term, wt in term_weights:
        if wt >= 0:
            d1.append((term, round(wt, 3)))
        else:
            d2.append((term, round(wt, 3)))
    print('Direction 1:', d1)
    print('-'*50)
    print('Direction 2:', d2)
    print('-'*50)
    print()

Topic #1:
Direction 1: [('training', 92.619), ('task', 80.732), ('pattern', 70.619), ('classifier', 56.987), ('control', 50.675), ('rule', 45.926), ('action', 41.202), ('neuron', 38.195)]
--------------------------------------------------
Direction 2: [('word', -188.486), ('vector', -85.973), ('node', -54.38), ('recognition', -53.231), ('sequence', -50.35), ('circuit', -45.396), ('cell', -44.811), ('hmm', -34.085), ('character', -34.022), ('chip', -32.162), ('matrix', -32.093), ('structure', -30.993)]
--------------------------------------------------

Topic #2:
Direction 1: [('node', 173.276), ('circuit', 92.999), ('chip', 73.593), ('classifier', 58.718), ('current', 55.844), ('voltage', 53.489), ('control', 51.709), ('rule', 45.295), ('layer', 40.265), ('analog', 38.343), ('tree', 33.483)]
--------------------------------------------------
Direction 2: [('word', -78.351), ('neuron', -69.792), ('stimulus', -63.233), ('feature', -53.819), ('distribution', -53.119), ('response', -30.954

In [34]:
document_topics = pd.DataFrame(np.round(topic_document.T, 3), 
                               columns=['T'+str(i) for i in range(1, TOTAL_TOPICS+1)])
document_numbers = [13, 250, 500]

for document_number in document_numbers:
    top_topics = list(document_topics.
                      columns[np.argsort(-np.absolute(document_topics.iloc[document_number].values))[:3]])
    print('Document #'+str(document_number)+':')
    print('Dominant Topics (top 3):', top_topics)
    print('Paper Summary:')
    print(papers[document_number][:500])
    print()

Document #13:
Dominant Topics (top 3): ['T3', 'T10', 'T7']
Paper Summary:
412 
CAPACITY FOR PATTERNS AND SEQUENCES IN KANERVA'S SDM 
AS COMPARED TO OTHER ASSOCIATIVE MEMORY MODELS 
James D. Keeler 
Chemistry Department, Stanford University, Stanford, CA 94305 
and RIACS, NASA-AMES 230-5 Moffett Field, CA 94035. 
e.rnail: jdk hydra.riacs. edu 
ABSTRACT 
The information capacity of Kanerva's Sparse, Distributed Memory (SDM) and Hopfield-type 
neural networks is investigated. Under the approximations used here, it is shown that the to- 
tal information stored in these s

Document #250:
Dominant Topics (top 3): ['T3', 'T10', 'T9']
Paper Summary:
68 Baird 
Associative Memory in a Simple Model of 
Oscillating Cortex 
Bill Baird 
Dept Molecular and Cell Biology, 
U.C.Berkeley, Berkeley, Ca. 94720 
ABSTRACT 
A generic model of oscillating cortex, which assumes "minimal" 
coupling justified by known anatomy, is shown to function as an 
sociative memory, using previously developed theory. The n